# Money Diaries Webscraping & Data Transformation 

Refinery29's "Money Diary" series features first-hand accounts from individuals detailing their personal financial information and spending habits. In this project, I have developed a web scraper to extract the raw, unformatted data from these published diaries. I then transform and clean the data before completing a thorough quantitative analysis, enabling deeper insights into the financial lives of the featured diarists.\
https://www.refinery29.com/en-us/money-diary


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os
import us
import time
import requests
import concurrent.futures

In [ ]:
# importing helper functions from Money_Diaries_helper.pynb
from Money_Diaries_helper import get_article_info
from Money_Diaries_helper import get_article_text
from Money_Diaries_helper import extract_salary
from Money_Diaries_helper import extract_state
from Money_Diaries_helper import get_monthly_expenses
from Money_Diaries_helper import get_age
from Money_Diaries_helper import get_occupation
from Money_Diaries_helper import get_industry
from Money_Diaries_helper import get_weekly_spend
from Money_Diaries_helper import get_mortgage_rent


### Scraping Titles and URLs

In [17]:
# Get article titles and URLs
article_title_url = []

for i in range(1, nb_of_pages + 1):
    result = get_article_info(i)
    if result is not None:
        article_title_url.append(result)
    else:
        print(f"Warning: No data returned for page {i}")

# Combine all DataFrames into one
if article_title_url:
    article_title_url = pd.concat(article_title_url, ignore_index=True)
    print("Successfully combined all pages.")
else:
    print("No data was retrieved from any page.")

article_title_url.head()

An error occurred on page 59: All arrays must be of the same length
Successfully combined all pages.


,url,title
0,https://www.refinery29.com/en-us/editor-washington-80k-money-diary,"A Week In Washington, D.C. On An $80,000 Salary"
1,https://www.refinery29.com/en-us/forensic-accountant-virginia-108k-money-diary,"A Week In Virginia On A $108,280 Salary"
2,https://www.refinery29.com/en-us/historic-preservation-specialist-hawaii-36k-money-diary,"A Week On O‘ahu, HI On A $36,000 Salary"
3,https://www.refinery29.com/en-us/project-manager-salt-lake-city-224k-joint-salary-money-diary,"A Week In Salt Lake City On A $224,000 Joint Salary"
4,https://www.refinery29.com/en-us/strategist-boston-81k-money-diary,"A Week In Boston On An $81,000 Salary"


### Processing Money Data
Sarlary, location, joint, hourly, monthly

In [19]:
# Process money data
money_data = article_title_url.copy()
money_data['lowercase_title'] = money_data['title'].str.lower()

money_data['salary'] = money_data['title'].apply(extract_salary)
money_data['location'] = money_data['title'].str.replace("A Week In ", "").str.replace("On .*", "", regex=True) ## o'ahu "a week on" 
money_data['joint'] = money_data['lowercase_title'].str.contains("joint")
money_data['hourly'] = money_data['lowercase_title'].str.contains("hour")
money_data['monthly'] = money_data['lowercase_title'].str.contains("month")

# Replace one broken URL
money_data['url'] = money_data['url'].replace(
    "https://www.refinery29.com/en-us/money-diary-wyoming-education-program-specialist-salary",
    "https://www.refinery29.com/en-gb/money-diary-wyoming-education-program-specialist-salary"
)

# fix for when location is the state 
money_data['state'] = money_data['location'].apply(extract_state)
money_data['city'] = money_data['location'].str.extract(r'^([^,]+)')

In [ ]:
# Filter by "week"
money_data = money_data[money_data['lowercase_title'].str.contains("week")]

money_data

### Scraping article info

In [ ]:
def fetch_article_text(url):
    return get_article_text(url)

# Split into two batches for processing
batch_size = 750
money_data_first = money_data.iloc[:batch_size].copy()
money_data_second = money_data.iloc[batch_size:].copy()

# Function to process batch
def process_batch(money_data_batch):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        article_texts = list(executor.map(fetch_article_text, money_data_batch['url']))
        print(article_texts)
    money_data_batch['article_text'] = article_texts
    return money_data_batch


# Process the first and second batches in parallel
all_article_text_first = process_batch(money_data_first)
all_article_text_second = process_batch(money_data_second)

# Combine batches
all_article_text = pd.concat([all_article_text_first, all_article_text_second], ignore_index=True)

csv_file_name = "scraped_article_text.csv"
all_article_text.to_csv(csv_file_name, index=False)

print(f"Data has been successfully saved to {csv_file_name}")


### Using the CSV scraped once for testing

In [24]:
#all_article_text = pd.read_csv("scraped_article_text.csv")

### Cleaning the data

In [26]:
print(all_article_text.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              1727 non-null   object 
 1   title            1727 non-null   object 
 2   lowercase_title  1727 non-null   object 
 3   salary           1684 non-null   float64
 4   location         1727 non-null   object 
 5   joint            1727 non-null   bool   
 6   hourly           1727 non-null   bool   
 7   monthly          1727 non-null   bool   
 8   state            1528 non-null   object 
 9   city             1727 non-null   object 
 10  article_text     1727 non-null   object 
dtypes: bool(3), float64(1), object(7)
memory usage: 113.1+ KB
None


In [27]:
# Drop all columns with names that start with 'Unnamed:'
all_article_text = all_article_text.loc[:, ~all_article_text.columns.str.contains('^Unnamed')]
print(all_article_text.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              1727 non-null   object 
 1   title            1727 non-null   object 
 2   lowercase_title  1727 non-null   object 
 3   salary           1684 non-null   float64
 4   location         1727 non-null   object 
 5   joint            1727 non-null   bool   
 6   hourly           1727 non-null   bool   
 7   monthly          1727 non-null   bool   
 8   state            1528 non-null   object 
 9   city             1727 non-null   object 
 10  article_text     1727 non-null   object 
dtypes: bool(3), float64(1), object(7)
memory usage: 113.1+ KB
None


In [28]:
def clean_text(text):
    # Flatten the list if it's a list of lists
    if isinstance(text, list):
        flattened_text = []
        for item in text:
            if isinstance(item, list):
                flattened_text.extend(item)
            else:
                flattened_text.append(item)

        # Convert everything to a string and remove HTML tags
        cleaned_text = ' '.join([BeautifulSoup(str(t), "html.parser").get_text() for t in flattened_text])
    else:
        # If it's not a list, just clean the text directly
        cleaned_text = BeautifulSoup(str(text), "html.parser").get_text()
    
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

In [29]:
all_article_text['article_text'] = all_article_text['article_text'].apply(clean_text)

In [30]:
print(all_article_text.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              1727 non-null   object 
 1   title            1727 non-null   object 
 2   lowercase_title  1727 non-null   object 
 3   salary           1684 non-null   float64
 4   location         1727 non-null   object 
 5   joint            1727 non-null   bool   
 6   hourly           1727 non-null   bool   
 7   monthly          1727 non-null   bool   
 8   state            1528 non-null   object 
 9   city             1727 non-null   object 
 10  article_text     1727 non-null   object 
dtypes: bool(3), float64(1), object(7)
memory usage: 113.1+ KB
None


In [ ]:
# Ensure 'lowercase_title' is a string and handle NaN values
#all_monthly_expenses['lowercase_title'] = all_monthly_expenses['lowercase_title'].fillna('').astype(str)

# Process expenses and filter data
all_monthly_expenses = all_article_text.copy()
all_monthly_expenses['monthly_expenses'] = all_monthly_expenses['article_text'].apply(get_monthly_expenses)

# Filter out rows where 'lowercase_title' contains specific terms
all_monthly_expenses = all_monthly_expenses[
    ~all_monthly_expenses['lowercase_title'].str.contains("couple's|couples|5 money diaries", regex=True)
]

print(all_monthly_expenses)


In [59]:
# Ensure 'lowercase_title' is a string and handle NaN values
all_monthly_expenses['lowercase_title'] = all_monthly_expenses['lowercase_title'].astype(str).fillna('')

# Verify the conversion
print(all_monthly_expenses['lowercase_title'].head())
print(all_monthly_expenses['lowercase_title'].dtype)


0        a week in washington, d.c. on an $80,000 salary
1                a week in virginia on a $108,280 salary
2                a week on o‘ahu, hi on a $36,000 salary
3    a week in salt lake city on a $224,000 joint salary
4                  a week in boston on an $81,000 salary
Name: lowercase_title, dtype: object
object


In [ ]:
try:
    # Apply the filter
    filtered_expenses = all_monthly_expenses[
        ~all_monthly_expenses['lowercase_title'].str.contains("couple's|couples|5 money diaries", regex=True)
    ]
    print(filtered_expenses)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
all_monthly_expenses = all_monthly_expenses[
    ~all_monthly_expenses['url'].str.contains("comparison")
]
# Apply the get_age function to the 'article_text' column
all_monthly_expenses['age'] = all_monthly_expenses['article_text'].apply(get_age)

print(all_monthly_expenses[['article_text', 'age']].sample(5))

In [ ]:
all_monthly_expenses['occupation'] = all_monthly_expenses['article_text'].apply(get_occupation)
print(all_monthly_expenses[['article_text', 'occupation']].sample(5))

In [71]:
all_monthly_expenses['industry'] = all_monthly_expenses['article_text'].apply(get_industry)
print(all_monthly_expenses[['industry']].sample(5))

              industry
491        Travel/Tech
1087             Legal
316   Higher Education
1544         Nonprofit
342      Public Health


In [73]:
all_monthly_expenses['total_weekly_spend'] = all_monthly_expenses['article_text'].apply(get_weekly_spend)
all_monthly_expenses['total_weekly_spend'].replace(0, pd.NA, inplace=True)
print(all_monthly_expenses[['total_weekly_spend']].sample(5))

     total_weekly_spend
37              1103.77
321              454.87
331              232.47
1142             343.78
500              343.31


In [75]:
all_monthly_expenses = all_monthly_expenses.dropna(subset=['state'])
all_monthly_expenses = all_monthly_expenses[all_monthly_expenses['state'].str.strip() != ''] 

all_monthly_expenses = all_monthly_expenses[all_monthly_expenses['url'].str.startswith('https://')]

all_monthly_expenses = all_monthly_expenses.loc[:, ~all_monthly_expenses.columns.str.contains('^Unnamed')]
print(all_monthly_expenses.info())

#all_monthly_expenses.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1523 entries, 0 to 1726
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   url                 1523 non-null   object 
 1   title               1523 non-null   object 
 2   lowercase_title     1523 non-null   object 
 3   salary              1497 non-null   float64
 4   location            1523 non-null   object 
 5   joint               1523 non-null   bool   
 6   hourly              1523 non-null   bool   
 7   monthly             1523 non-null   bool   
 8   state               1523 non-null   object 
 9   city                1523 non-null   object 
 10  article_text        1523 non-null   object 
 11  monthly_expenses    1520 non-null   object 
 12  age                 1523 non-null   float64
 13  occupation          1487 non-null   object 
 14  industry            1507 non-null   object 
 15  total_weekly_spend  1420 non-null   object 
dtypes: bool(3),

In [77]:
csv_file_name = "scraped_age_thru_industry.csv"
all_monthly_expenses.to_csv(csv_file_name, index=False)

print(f"Data has been successfully saved to {csv_file_name}")

Data has been successfully saved to scraped_age_thru_industry.csv


In [79]:
def get_mortgage_rent(expenses):
    expenses_df = pd.DataFrame(expenses, columns=['Type', 'Amount'])
    rent_mortgage = expenses_df[expenses_df['Type'].str.contains("Rent|Mortgage", case=False, na=False) & 
                                ~expenses_df['Type'].str.contains("insurance", case=False, na=False)]
    return rent_mortgage['Amount'].astype(float).tolist()

In [81]:
import pandas as pd

# Example of the get_mortgage_rent function
def get_mortgage_rent(expenses):
    expenses_df = pd.DataFrame(expenses, columns=['Type', 'Amount'])
    rent_mortgage = expenses_df[expenses_df['Type'].str.contains("Rent|Mortgage", case=False, na=False) & 
                                ~expenses_df['Type'].str.contains("insurance", case=False, na=False)]
    return rent_mortgage['Amount'].astype(float).tolist()

# Split non-null and null expenses
non_null_expenses = all_monthly_expenses.dropna(subset=['monthly_expenses'])
null_expenses = all_monthly_expenses[all_monthly_expenses['monthly_expenses'].isna()]

# Explode the 'monthly_expenses' list and split into 'Type' and 'Amount'
share_for_housing = non_null_expenses.explode('monthly_expenses').copy()
share_for_housing[['Type', 'Amount']] = share_for_housing['monthly_expenses'].str.split(':', expand=True)
share_for_housing['Amount'] = share_for_housing['Amount'].str.replace(",", "").str.extract(r'(\d+)').astype(float)

# Ensure 'salary' column is numeric
share_for_housing['salary'] = pd.to_numeric(share_for_housing['salary'], errors='coerce')

# Apply get_mortgage_rent to each group of rows corresponding to the same 'id'
if 'id' not in share_for_housing.columns:
    share_for_housing['id'] = share_for_housing.index  # Create 'id' only if it doesn't exist

# Group by 'id' and apply the get_mortgage_rent function to get the rent or mortgage amount
share_for_housing['rent_mortgage'] = share_for_housing.groupby('id').apply(lambda x: get_mortgage_rent(list(zip(x['Type'], x['Amount'])))).reset_index(level=0, drop=True)

# Check if 'rent_mortgage' contains lists and filter out rows where it does not
share_for_housing = share_for_housing[share_for_housing['rent_mortgage'].apply(lambda x: isinstance(x, list) and len(x) == 1)]

# Convert 'rent_mortgage' to float (since it will have exactly one element)
share_for_housing['rent_mortgage'] = share_for_housing['rent_mortgage'].apply(lambda x: float(x[0]) if isinstance(x, list) else None)

# Calculate the housing share
share_for_housing['housing_share'] = share_for_housing.apply(
    lambda row: row['rent_mortgage'] / row['salary'] if row['monthly'] else
    row['rent_mortgage'] / (row['salary'] * 40 * 4) if row['hourly'] else
    (row['rent_mortgage'] * 12) / row['salary'], axis=1
)

# Drop duplicates if any exist due to the previous operations
share_for_housing = share_for_housing.drop_duplicates(subset=['id'])

#print(share_for_housing)


In [83]:
# Save the final data
csv_file_name = "final_money_diaries.csv"
share_for_housing.to_csv(csv_file_name, index=False)

print(f"Data has been successfully saved to {csv_file_name}")

Data has been successfully saved to final_money_diaries.csv
